# Basic Profiling

A simple way to estimate the environmental impact of our code is to make use of functions from python's `time`, `psutil` and `os` modules. 

We can easily measure the total runtime. 

Although memory allocation in python is handled by the garbage collector, we can still get a very rough idea of the memory used for the operation by recording the total memory before and after.


In [62]:
import psutil, gc, os, time

# Function to get memory usage
def get_memory_usage():
    process = psutil.Process(os.getpid())
    mem_info = process.memory_info()
    return mem_info.rss / 1024 / 1024  # Memory in MB

def fib(n):
    if(n == 0):
        return 0
    if(n == 1):
        return 1
    return fib(n - 1) + fib(n - 2)

def fibs(n):
    return [fib(i) for i in range(n)]

# Clear the memory before starting
gc.collect() 

mem_before = get_memory_usage()
time_before = time.time()

print(fibs(35))

mem_after = get_memory_usage()
time_after = time.time()

print(f"execution time: {(time_after - time_before):,.4f}s")
print(f"memory diff: {(mem_after - mem_before):,.4f}M")


[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711, 28657, 46368, 75025, 121393, 196418, 317811, 514229, 832040, 1346269, 2178309, 3524578, 5702887]
execution time: 1.6275s
memory diff: 0.0000M


## Task 1: Memoization

[Memoization](https://en.wikipedia.org/wiki/Memoization) is a simple technique to avoid repeated calculations.

A memoised version of the `fib()` function would massively reduce the required runtime.

Write one below and see how it changes the runtime and memory usage.

In [60]:
_fib = {0: 0, 1: 1}

def fib_memo(n):
    if n in _fib:
        return _fib[n]
    else:
        result = fib_memo(n - 1) + fib_memo(n - 2)
        _fib[n] = result
        return result
    
def fibs_memo(n):
    return [fib_memo(i) for i in range(n)]


In [61]:
gc.collect() 

mem_before = get_memory_usage()
time_before = time.time()

print(fibs_memo(35))

mem_after = get_memory_usage()
time_after = time.time()

print(f"execution time: {(time_after - time_before):,.4f}s")
print(f"memory diff: {(mem_after - mem_before):,.4f}M")

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711, 28657, 46368, 75025, 121393, 196418, 317811, 514229, 832040, 1346269, 2178309, 3524578, 5702887]
execution time: 0.0003s
memory diff: 0.0000M


## Task 2: Pathfinding

Here we will use a graph theory problem as an example of a task that can be accomplished in different ways.

A *[graph](https://en.wikipedia.org/wiki/Graph_(discrete_mathematics)#Undirected_graph)* is a set of *vertices* plus a set of *edges* (vertex pairs) that connect some of the vertices.
We will consider only undirected graphs.

The file `data/facebook_combined.txt` contains a graph of friends taken from Facebook (4,039 nodes and 88,234 edges). Each person is a vertex, labelled with a number.

(You can find more information about this dataset at https://snap.stanford.edu/data/ego-Facebook.html)

Find the shortest path from person 0 to person 4038.

You will probably find the package [NetworkX](https://networkx.org/documentation/stable/index.html) useful.



In [2]:
import networkx as nx

file_path = "data/facebook_combined.txt"
G = nx.read_edgelist(file_path, nodetype=int)

### (1) Using a brute force method

In [ ]:
# function to find all the possible extensions of a path
def extend(G, path):
    ex = []
    for v in G.neighbors(path[-1]):
        if v not in path:
            ex.append( path + [v] )
    return ex

# brute force pathfinding
def path_brute(G, start, end):

    # initialise paths
    P = [ [start] ]   # stores all paths originating at `start`

    # loop
    finished = False
    while(not finished):
        finished = True
        P_next = []
        for p in P:
            ex = extend(G, p)
            for p2 in ex:
                
                if p2[-1] == end:
                    return p2                  # we have a solution!
                else:
                    finished = False           # this path needs to be processed further
                    P_next.append(p2)
        P = P_next


path_brute(G, 0, 4038)

[0, 34, 414, 594, 3980, 4038]

### (2) Using a more efficient algorithm

In [ ]:
# Using the Dijkstra algorithm implemented in NetworkX
nx.shortest_path(G, 0, 4038)  

[0, 34, 414, 594, 3980, 4038]

### Compare the runtime and memory usage of your two versions.

In [54]:
def task_brute():
    return path_brute(G, 0, 4038)

def task_alg():
    return nx.shortest_path(G, 0, 4038)

def report(desc, task):

    print(desc)

    gc.collect() 

    mem_before = get_memory_usage()
    time_before = time.time()

    print(task())

    mem_after = get_memory_usage()
    time_after = time.time()

    print(f"execution time: {(time_after - time_before):,.4f}s")
    print(f"memory diff: {(mem_after - mem_before):,.4f}M")
    print()


In [55]:
report("brute force", task_brute)

brute force
[0, 34, 414, 594, 3980, 4038]
execution time: 21.3086s
memory diff: -30.4688M



In [56]:
report("dijkstra algorithm", task_alg)

dijkstra algorithm
[0, 34, 414, 594, 3980, 4038]
execution time: 0.0001s
memory diff: 0.0000M



## Task 3: More robust estimates

You will have noticed that there can be big variations in both the runtime and the memory usage.

Write a wrapper function `report_stats()` that accepts a function (the operation to be profiled) and an integer (the number of times to repeat the profiling). 

Your function should report the mean and standard deviation for both the runtime and memory usage.

In [19]:
import numpy as np

def report_stats(desc, task, n_iter):

    print(desc)

    results = []
    runtimes = []
    mem_usages = []

    for i in range(n_iter):

        gc.collect() 

        mem_before = get_memory_usage()
        time_before = time.time()

        results.append(task())

        mem_after = get_memory_usage()
        time_after = time.time()

        runtimes.append(time_after - time_before)
        mem_usages.append(mem_after - mem_before)

    print(f"mean execution time: {np.mean(runtimes):,.4f}s")
    print(f"std execution time: {np.std(runtimes):,.4f}s")
    print(f"mean memory usage: {np.mean(mem_usages):,.4f}M")
    print(f"std memory usage: {np.std(mem_usages):,.4f}M")
    print()

    return results


In [21]:
report_stats("brute force", task_brute, 10)

brute force
mean execution time: 19.6116s
std execution time: 1.8628s
mean memory usage: -62.3000M
std memory usage: 6.5450M



[[0, 34, 414, 594, 3980, 4038],
 [0, 34, 414, 594, 3980, 4038],
 [0, 34, 414, 594, 3980, 4038],
 [0, 34, 414, 594, 3980, 4038],
 [0, 34, 414, 594, 3980, 4038],
 [0, 34, 414, 594, 3980, 4038],
 [0, 34, 414, 594, 3980, 4038],
 [0, 34, 414, 594, 3980, 4038],
 [0, 34, 414, 594, 3980, 4038],
 [0, 34, 414, 594, 3980, 4038]]

In [20]:
report_stats("dijkstra algorithm", task_alg, 10)

dijkstra algorithm
mean execution time: 0.0001s
std execution time: 0.0000s
mean memory usage: 0.0063M
std memory usage: 0.0143M



[[0, 34, 414, 594, 3980, 4038],
 [0, 34, 414, 594, 3980, 4038],
 [0, 34, 414, 594, 3980, 4038],
 [0, 34, 414, 594, 3980, 4038],
 [0, 34, 414, 594, 3980, 4038],
 [0, 34, 414, 594, 3980, 4038],
 [0, 34, 414, 594, 3980, 4038],
 [0, 34, 414, 594, 3980, 4038],
 [0, 34, 414, 594, 3980, 4038],
 [0, 34, 414, 594, 3980, 4038]]

## Task 4: More detailed profiling


Profiling by function gives us a high-level idea of how often functions are called and how long those calls last. One way to do this is to import the `cProfile` module and run a function using the `cProfile.run()` function, providing a string argument which is the command used to invoke the function. `cProfile` is part of the Python standard library and so is available without installing any additional packages. For example:

In [63]:
import cProfile

# Call the cProfile.run() method with a string argument that is the call to the function you want to profile
cProfile.run('fibs(35)', sort='tottime')

         48316336 function calls (758 primitive calls) in 5.379 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
48315597/35    5.378    0.000    5.438    0.155 419126109.py:9(fib)
      7/3    0.000    0.000    0.002    0.001 events.py:87(_run)
        5    0.000    0.000    0.000    0.000 attrsettr.py:66(_get_attr_opt)
       14    0.000    0.000    0.000    0.000 socket.py:623(send)
       66    0.000    0.000    0.000    0.000 enum.py:1589(_get_value)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        2    0.000    0.000    0.000    0.000 socket.py:771(recv_multipart)
       15    0.000    0.000    0.000    0.000 enum.py:1596(__or__)
        6    0.000    0.000    0.000    0.000 typing.py:426(inner)
        5    0.000    0.000    0.000    0.000 attrsettr.py:43(__getattr__)
  143/139    0.000    0.000    0.000    0.000 {built-in method builtins.isinstance}
        1    0

The output shows the total time spent running the code and the total number of function calls. Then, for each function, it shows:
* ```ncalls```: the number of times the function was called.
* ```tottime```: the total time spent in the function, excluding time spent in functions called by the function.
* ```percall```: the time spent in the function per call, excluding time spent in functions called by the function.
* ```cumtime```: the total time spent in the function, including time spent in functions called by the function.
* ```percall```: the time spent in the function per call, including time spent in functions called by the function.
* ```filename:lineno(function)```: the filename, line number and function name.

In the output you will see a number of functions which you have neither written nor explicitly called. These are often called as part of how Python internally executes your code. They are normally not very consequential in terms of run-time and can often be ignored.

Try using `cProfile.run()` to evaluate your pathfinding code.

In [64]:
cProfile.run('task_brute()', sort='tottime')

         71643636 function calls (71643611 primitive calls) in 26.039 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   802603   14.081    0.000   16.213    0.000 442339266.py:2(extend)
      6/1    5.327    0.888    0.000    0.000 {method 'control' of 'select.kqueue' objects}
 68908812    3.815    0.000    3.815    0.000 {method 'append' of 'list' objects}
      6/1    1.027    0.171    0.006    0.006 selectors.py:540(select)
        1    0.984    0.984    3.617    3.617 3970698551.py:1(task_brute)
        1    0.559    0.559    2.633    2.633 442339266.py:10(path_brute)
   802603    0.115    0.000    0.156    0.000 graph.py:1333(neighbors)
    34338    0.044    0.000    0.082    0.000 ipkernel.py:775(_clean_thread_parent_frames)
   802605    0.040    0.000    0.040    0.000 {built-in method builtins.iter}
    17169    0.013    0.000    0.018    0.000 threading.py:1479(enumerate)
   120183    0.012    0.000    0.012    0.

In [33]:
cProfile.run('task_alg()', sort='tottime')

         690 function calls (689 primitive calls) in 0.001 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.001    0.001 unweighted.py:269(_bidirectional_pred_succ)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
       62    0.000    0.000    0.000    0.000 coreviews.py:81(__getitem__)
        1    0.000    0.000    0.001    0.001 <class 'networkx.utils.decorators.argmap'> compilation 26:1(argmap_bidirectional_shortest_path_23)
      417    0.000    0.000    0.000    0.000 {method 'append' of 'list' objects}
        1    0.000    0.000    0.001    0.001 <class 'networkx.utils.decorators.argmap'> compilation 22:1(argmap_shortest_path_19)
        1    0.000    0.000    0.001    0.001 3970698551.py:4(task_alg)
      2/1    0.000    0.000    0.001    0.001 backends.py:959(__call__)
       62    0.000    0.000    0.000    0.000 coreviews.py:50(__iter

---